# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_cities_data = pd.read_csv("Output_Data/cities.csv")
weather_cities_data


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
humidity = weather_cities_data["Humidity"].astype(float)
locations = weather_cities_data[["Lat", "Lng"]]

In [5]:
google_heat_map = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=40,
                               point_radius =1.5)
google_heat_map.add_layer(heat_layer)
google_heat_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Create data frames by filtering to your criteria; created three data frames
perfect_temperate_df=weather_cities_data.loc[(weather_cities_data['Max Temp']>=60)&(weather_cities_data['Max Temp']<=80)]
wind_df=perfect_temperate_df.loc[(perfect_temperate_df['Wind Speed']<=8)]
perfect_weather_city=wind_df.loc[(wind_df['Cloudiness']==0)]

perfect_weather_city.sort_values(by='Country', ascending=False)


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78
433,433,svodin,0,SK,1558378878,72,47.91,18.50,63.00,6.93
508,508,noumea,0,NC,1558378811,77,-22.28,166.46,64.40,2.24
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24
298,298,mikkeli,0,FI,1558378844,59,61.69,27.27,62.60,6.93
533,533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82
371,371,carballo,0,ES,1558378860,44,43.21,-8.69,63.00,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:

hotel_df = perfect_weather_city.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

#Create for loop and parameters
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)

    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=3, sort_keys=True))
    
    #Include try to get rid of the errors
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Skipping Missing Field")
hotel_df

{
   "html_attributions": [],
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -18.0446824,
               "lng": 13.8335005
            },
            "viewport": {
               "northeast": {
                  "lat": -18.04348247010728,
                  "lng": 13.83472832989272
               },
               "southwest": {
                  "lat": -18.04618212989272,
                  "lng": 13.83202867010728
               }
            }
         },
         "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
         "icon_background_color": "#909CE1",
         "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
         "name": "Opuwo Country Lodge",
         "photos": [
            {
               "height": 2592,
               "html_attributions": [
                  "<a href=\"https://maps.google.com/maps/contri

{
   "html_attributions": [],
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -22.3906663,
               "lng": 18.6153764
            },
            "viewport": {
               "northeast": {
                  "lat": -22.38871372010728,
                  "lng": 18.61680377989272
               },
               "southwest": {
                  "lat": -22.39141337989272,
                  "lng": 18.61410412010728
               }
            }
         },
         "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
         "icon_background_color": "#909CE1",
         "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
         "name": "West Nest Lodge",
         "photos": [
            {
               "height": 3120,
               "html_attributions": [
                  "<a href=\"https://maps.google.com/maps/contrib/10

{
   "html_attributions": [],
   "next_page_token": "Aap_uECLmv2RW0BUvTQf7ofa7xLdp7raNFCEH5Ykgj0yrRWxUBVUxuEPRM9LU2wExoQpMwp9vni8odvvWyBpUl3kOyX53OOiz9ntwiwEbQo0IhvkewTdWkqbQ34SlImj56Y6bvBlNsbK-Jx0okzP43W-k-5KFTuR67jHe3KTWTspf5nh5vDTnxrJyVqSI6D9QGGOcnAgINbuICEYSVUID7OsdSiPYcO3bvUd-2VrtI8dSkMOqw5zMlcvGSeHYEk6vwB0qAq6LHo9cZQLiGWp7NVIPfUEXGHiLBoNs2P6ixNPOEKVHxO2Cq8J9aOsmGcsifTx7olBkMGvAnlnkc8o4yHQ0GOM6GKv9kvu9EVKQhtKtuy_7iK8OUk1_-h2jERiWmkyZu8USII6zmpwj5gKmOVrapcU_hnI7kbUxXhIO1H1iuA7SvoHCw0b9UpcBnYqiV0a",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -20.6441728,
               "lng": -46.1058822
            },
            "viewport": {
               "northeast": {
                  "lat": -20.64274842010728,
                  "lng": -46.10453982010728
               },
               "southwest": {
                  "lat": -20.64544807989272,
                  "lng": -46.10723947989273
          

{
   "html_attributions": [],
   "next_page_token": "Aap_uEAskC5e2FtjL9a8u6H7pgDqIgS5Uv3zbRprhVmAr0dzV_iBhheCtjcQaJddojseA3UXSJEGuRP_dbLRgtWCtq3nc6DWRq1OAo1sz-SmKJUGcvyL0PsxhPaOp6yPV2jceKSfbu9x_AXrhveOofDV1GLIlBaxhXbINbLPizVRjTKAQfk52k8rLa590QKskHNrEmzAIIxv9GHiFWM0ALflPZp_uc7W3OuaIBr_C4wiRAvGVfVvkelUxoqRfsKlMaYspAhwfFEQaw_PFcuGR77f_e91hOFizksQ6XTGl9LNq_yeGLUjfz-LeIMGWMHketTA-Ng76RwoN_tkEGGWNIwg0piVHi8yxsnGJ0Guk1BWLvL2S1xyBuivrLWosL6_RMgxbcHCwaYDBddwJOpTMR-Swdjm_Rsa7jBpSMcYDvvQiNcvhMvN0ryoZScwb0GouTqk",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -23.5418436,
               "lng": 148.1730014
            },
            "viewport": {
               "northeast": {
                  "lat": -23.54049587010728,
                  "lng": 148.1743681798927
               },
               "southwest": {
                  "lat": -23.54319552989272,
                  "lng": 148.1716685201073
            

{
   "html_attributions": [],
   "next_page_token": "Aap_uEB0p0SqoEh9_ZIljpm48PCCR6g4RrawrIQDREB6W5cc2azyOx08dwS3NdgTZKBGncfYARK2f4bO8V9RGJCcTfhw-ndPnzHR30-Q4DxU0SRmYR9tAw7PSfwquFL2dQdJlci5LNVRh_Myrdtr7UvcnIA5bqSO6Z88rxkh5n_6eK990OskAIJ0N5uCe5rE5DEWl9CaJWJt7ZRFlPKngs6VYvC5Ki7QXoKHW_M2LEtyGHn2G_xz3bOkbqvnRQoW2x-8S-q68CUnMhd_bAwQ3rsDNZJS8WPk_OqHUVad4s6SxpRXnpYGSXDYdVnvNeNw7NI-C5zPem01JZAxZRhbI3H3FkSmU7MgVFI4ZC0gzyCWcF2maB_1wnCLwWj6VzzA7Mo3IQJ6EI2UyGbtGpMRysKNAd51DIOrTpe5yP3qD9W1ibModFz9fn3wuk7RZ5xhFCTL",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -29.7413463,
               "lng": -50.0030861
            },
            "viewport": {
               "northeast": {
                  "lat": -29.73992642010727,
                  "lng": -50.00188702010728
               },
               "southwest": {
                  "lat": -29.74262607989272,
                  "lng": -50.00458667989272
          

...

{
   "html_attributions": [],
   "next_page_token": "Aap_uEA9Hgegoz4XeFsOQ9C8Ja0EUuCO-uSTNi0CPko-VAD3OQztCyL2dMlVbQYmbh0nUCesheyJdQJLmwu5U38KLd1n5K_PssCYAhLJCCaPH0m9SqoEBtv3kiUZ07c8r1FI8yKH8cx3i231f49CtUI2gUydZ2bYI3VBECz9zv4Ma-cjUnw0XGruX2K5QN_T086UpTaykeesrvVM3HP5RlCglA7xAWwLvoIY6uBJsaienYMMIWSO1PVWdDYDm8QIjJRhfwl0BMqK7ttMJqPkyj9PKNtIMzSkZBpPFDLGCXm84PNxzcevQSbeRHEKfpnUWInS0uigOggPVqpHDiPQ6fTID14snpC0JJjBWPAOX389UnAJq2HIp1LdS1GC9okQQoxHcDObc5uDr76H4uByLo1PgPpmqIlZ2-wBfEub7stkhvtntRn30XTDVw2II_Ip2X5q",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -24.811201,
               "lng": 152.4615936
            },
            "viewport": {
               "northeast": {
                  "lat": -24.80979357010728,
                  "lng": 152.4630594798927
               },
               "southwest": {
                  "lat": -24.81249322989272,
                  "lng": 152.4603598201072
             

{
   "html_attributions": [],
   "next_page_token": "Aap_uEB-vnwDuAwasiNs6NTxo7uLpmIZXOMIAqst7pOkJ1LW-5NnkdOLhKtjYoR72_Px4wMQN_xU6EZVwjqjMJmonUX_e5DHTnBh3TlwTH1CVud50PTG5X59FY7geKWvHwrqdq2QZ2M7YPCoj7M9YyCagpRDC-eHCK8dgZgpc-RC38mbKXa950rirzqjgagVBr87CfLuGTDw3NoI0dmra4KwR2f6IWKb7hbWnDAhmSuMHkycvPMD6ueTiCRr-5__WWdvqvF9LAmK9AFotz0IyggQ-qY1bgcB56V7rQPVtTZLmu_6Wds3eI34bnotQXktUULAd1JJc6bfSsfFkT3DpOgbE6gEutNXLP14CPjPknHQWNXQzpVO7NxmZKeiVVEtA8OB-TxDkCytnAhVMZAbhgbuhZyZ16ha7vr46MPOcYCejnvdM-xLERkYdnWSPmg0CjSL",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": -22.3335063,
               "lng": 166.4101131
            },
            "viewport": {
               "northeast": {
                  "lat": -22.33215647010728,
                  "lng": 166.4114629298927
               },
               "southwest": {
                  "lat": -22.33485612989272,
                  "lng": 166.4087632701072
            

{
   "html_attributions": [],
   "next_page_token": "Aap_uEDFmdmSdbAlygLXz4EyDxiciVurixkQLp-jg6bt_LrXXWmrEaWugTsLDCzeayvvV8bRil6_m-rlK9iVI2NsRuirDVHIC1U0Y0DuglNvOhVoPGw9iBV2xN2iszz_G-Nj49BxBvYq5He9odNJYkIv9V0rf4jQpeOsRzTPVbJFgFeKLMRO6Zp31mf4EgPKIjXoHuj8rMErGZczh3WBUcYarujeu4uc7D_0BFvW5vb7xBIrm3m28wOFNOtGEhskwnds8K5UO5PYlTPHQ2_NZkpDDMeVfg1S5_tFQqWnG4C-vgTbeSj6A5JfUH9cT54IkaPzCUg6v08RgN-mUcZxZr09B0MDy6PMzjRTg4mzK38dDuHUfwWE2sylY5wL1EU86NTOZ7DHRpkQaDScSpuCC-RMHkh9uf9dIlw2gNMH201u1o4nPtBDFS_CZW5wKiKor8-Y",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": 61.15855000000001,
               "lng": 24.3213001
            },
            "viewport": {
               "northeast": {
                  "lat": 61.15993157989272,
                  "lng": 24.32271187989272
               },
               "southwest": {
                  "lat": 61.15723192010727,
                  "lng": 24.32001222010728
         

{
   "html_attributions": [],
   "next_page_token": "Aap_uEBNIbI1zhbi36fMEu45aMWEqWhLUaAi1aEIbEgnFyjzRRBEchhphhtcso7aqSdAC3-46zoDn3TOrCpQGrhSB-WnyYK9cUWpetqdJp1Dqr2O7_LPmjCnR29V-jqDX4a8Horb2ZBSDiy4Ulehy-IfnPzkVMd9DEeI0l0MlLHETbMQ09frH0JlHcFujBZI_7V44TtR0my7HZd8FaH-odjt5VZnaU5eDXOgfEzERKPtskXV_PdLEbP8hnVwD8bPojOGCFwxRQAGD_rcnIbdOU37vImItttEiL28DG4KPPoc26iB3uEvPYFt1d4L4fTuaaLGg2eHUEZeevvTu9eP5ZdbnAbJvc5TuIu3wyP1BAnQcd9ZiQW7fe6bj76xcJETWutMreRlhcqkp1usirBr6GFIvU6LtTHIkAo6IcJhzckZw2yEDpU-iRvbjxZImOJWMMEY",
   "results": [
      {
         "business_status": "OPERATIONAL",
         "geometry": {
            "location": {
               "lat": 43.07875749999999,
               "lng": 5.8916996
            },
            "viewport": {
               "northeast": {
                  "lat": 43.08010732989272,
                  "lng": 5.893049429892722
               },
               "southwest": {
                  "lat": 43.07740767010728,
                  "lng": 5.890349770107278
          

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Opuwo Country Lodge
1,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24,West Nest Lodge
2,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Balneário do Lago Hotel
3,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93,EMERALD EXPLORERS INN
4,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Araçá
5,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,April20 Villas by the Beach
6,141,sanchazi,0,CN,1558378781,82,30.43,103.62,64.40,2.24,The Temple House
7,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Mercure Rif Nador
8,192,port hedland,0,AU,1558378791,51,-20.31,118.58,62.60,6.93,Hedland Hotel
9,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93,Isa Hotel


In [11]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [12]:
markers = gmaps.marker_layer(hotel_locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

google_heat_map.add_layer(markers)
google_heat_map


Figure(layout=FigureLayout(height='420px'))